# Debug centering issue

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from hcipy.optics.segmented_mirror import SegmentedMirror

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
# Instantiate LUVOIR
apodizer_design = 'small'
sampling = 4
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf_unaber, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

In [ ]:
inter.keys()

In [ ]:
to_plot = inter['before_lyot']

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(2, 2, 1)
plt.title("Intensity")
hc.imshow_field(np.log10(to_plot.intensity))

plt.subplot(2, 2, 2)
plt.title("Phase")
hc.imshow_field(to_plot.phase)

plt.subplot(2, 2, 3)
plt.title("Real")
hc.imshow_field(to_plot.real)

plt.subplot(2, 2, 4)
plt.title("Imaginary")
hc.imshow_field(to_plot.imag)

In [ ]:
(np.arange(10) + 0.5 - 10/2)

In [ ]:
plt.figure(figsize=(10,10))
hc.imshow_field(luvoir.fpm)
plt.grid(color='w', linestyle='-', linewidth=2)

In [ ]:
res = util.FFT(luvoir.fpm.shaped)

In [ ]:
new_plot = res

im = util.zoom_point(new_plot, new_plot.shape[0]/2, new_plot.shape[0]/2, 10)

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(2, 2, 1)
plt.title("Intensity")
plt.imshow(np.abs(im)**2)

plt.subplot(2, 2, 2)
plt.title("Phase")
plt.imshow(np.angle(new_plot))
plt.colorbar()

plt.subplot(2, 2, 3)
plt.title("Real")
#hc.imshow_field(to_plot.real)

plt.subplot(2, 2, 4)
plt.title("Imaginary")
#hc.imshow_field(to_plot.imag)

In [ ]:
# Plot
plt.figure(figsize=(18, 6))
plt.subplot(131)
hc.imshow_field(psf_unaber/norm, norm=LogNorm())
plt.subplot(132)
hc.imshow_field(dh_mask)
plt.subplot(133)
hc.imshow_field(psf_unaber/norm, norm=LogNorm(), mask=dh_mask)